# Setup Python Environment
The next cell sets up the dependencies in required for the notebook, run it.

In [1]:
# Setup environment
!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install tensorflow==2.10.0-rc1
print("done")

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
done


# Train Neural Network
## Parse and prepare the data
The next cell parses the csv files and transforms them to a format that will be used to train the full connected neural network.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import fileinput
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Ftrl

import time

print(f"TensorFlow version = {tf.__version__}\n")

# Set a fixed random seed value, for reproducibility, this will allow us to get
# the same random numbers each time the notebook is run

2023-02-16 13:27:28.912246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 13:27:28.974555: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-16 13:27:28.992548: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-16 13:27:29.353628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

TensorFlow version = 2.10.0-rc1



In [3]:
# SEED = 1337
# np.random.seed(SEED)
# tf.random.set_seed(SEED)

# SEED=int(time.time())
# np.random.seed(SEED)
# tf.random.set_seed(SEED)

CLASSES = [];

for file in os.listdir("../data/own-csv"):
    if file.endswith(".csv"):
        CLASSES.append(os.path.splitext(file)[0])

CLASSES.sort()

print(CLASSES)

['laptop', 'phone_charging', 'school_box', 'school_pc', 'school_printer']


In [4]:
SAMPLES_WINDOW_LEN = 1
NUM_CLASSES = len(CLASSES)

# create a one-hot encoded matrix that is used in the output
ONE_HOT_ENCODED_CLASSES = np.eye(NUM_CLASSES)

inputs = []
outputs = []

# read each csv file and push an input and output
for class_index in range(NUM_CLASSES):
  objectClass = CLASSES[class_index]
  df = pd.read_csv("../data/own-csv/" + objectClass + ".csv") #, parse_dates=['timestamp']
  df = df.drop(columns=['time'])
  df = df.drop(columns=['Voltage'])
  columns = list(df)
  # get rid of pesky empty value lines of csv which cause NaN inputs to TensorFlow
  df = df.dropna()
  df = df.reset_index(drop=True)

  # calculate the number of objectClass recordings in the file
  num_recordings = int(df.shape[0] / SAMPLES_WINDOW_LEN)
  print(f"\u001b[32;4m{objectClass}\u001b[0m class will be output \u001b[32m{class_index}\u001b[0m of the classifier")
  print(f"{num_recordings} samples captured for training with inputs {list(df)} \n")
  
  #tensors
  output = ONE_HOT_ENCODED_CLASSES[class_index]
  for i in range(num_recordings):
    tensor = []
    row = []
    for c in columns:
      row.append(df[c][i])
    tensor += row
    inputs.append(tensor)
    outputs.append(output)

# convert the list to numpy array
inputs = np.array(inputs)
outputs = np.array(outputs)
num_inputs_2 = len(inputs)
print(CLASSES)
print(inputs)
print(outputs)
print(num_inputs_2)

# print("Data set parsing and preparation complete.")
# startTimestamp = df['timestamp'].iloc[0]
# endTimestamp = df['timestamp'].iloc[-1]
# print(endTimestamp - startTimestamp)


laptop class will be output 0 of the classifier
119 samples captured for training with inputs ['ApparentPower', 'Current', 'Factor', 'Power', 'ReactivePower'] 

phone_charging class will be output 1 of the classifier
209 samples captured for training with inputs ['ApparentPower', 'Current', 'Factor', 'Power', 'ReactivePower'] 

school_box class will be output 2 of the classifier
124 samples captured for training with inputs ['ApparentPower', 'Current', 'Factor', 'Power', 'ReactivePower'] 

school_pc class will be output 3 of the classifier
123 samples captured for training with inputs ['ApparentPower', 'Current', 'Factor', 'Power', 'ReactivePower'] 

school_printer class will be output 4 of the classifier
119 samples captured for training with inputs ['ApparentPower', 'Current', 'Factor', 'Power', 'ReactivePower'] 

['laptop', 'phone_charging', 'school_box', 'school_pc', 'school_printer']
[[55.     0.25   0.93  51.    20.   ]
 [52.     0.239  1.    52.     0.   ]
 [54.     0.247  0.96 

In [5]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation
# https://stackoverflow.com/a/37710486/2020087
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]

# Split the recordings (group of samples) into three sets: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.")
print(inputs_test)
print(num_inputs)

Data set randomization and splitting complete.
[[1.70e+01 7.70e-02 5.60e-01 1.00e+01 1.40e+01]
 [9.90e+01 4.52e-01 9.80e-01 9.70e+01 1.90e+01]
 [1.70e+01 7.80e-02 5.70e-01 1.00e+01 1.40e+01]
 [1.10e+01 5.20e-02 4.50e-01 5.00e+00 1.00e+01]
 [2.50e+01 1.15e-01 5.60e-01 1.40e+01 2.10e+01]
 [1.00e+02 4.55e-01 9.80e-01 9.80e+01 2.00e+01]
 [3.10e+01 1.43e-01 8.40e-01 2.60e+01 1.70e+01]
 [3.10e+01 1.39e-01 8.70e-01 2.70e+01 1.50e+01]
 [1.30e+01 5.80e-02 4.70e-01 6.00e+00 1.10e+01]
 [2.50e+01 1.16e-01 5.70e-01 1.40e+01 2.10e+01]
 [4.20e+01 1.94e-01 7.20e-01 3.00e+01 2.90e+01]
 [4.20e+01 1.93e-01 7.30e-01 3.10e+01 2.80e+01]
 [1.10e+01 5.20e-02 4.40e-01 5.00e+00 1.00e+01]
 [1.10e+01 5.20e-02 4.40e-01 5.00e+00 1.00e+01]
 [2.60e+01 1.17e-01 5.60e-01 1.40e+01 2.10e+01]
 [5.00e+00 2.30e-02 1.60e-01 1.00e+00 5.00e+00]
 [1.20e+01 5.30e-02 2.50e-01 3.00e+00 1.10e+01]
 [3.40e+01 1.55e-01 8.00e-01 2.70e+01 2.10e+01]
 [2.50e+01 1.15e-01 5.70e-01 1.40e+01 2.10e+01]
 [2.40e+01 1.11e-01 5.70e-01 1.40e+01 2.0

# Test code voor de tijd in de csv om te zetten

#Build & Train the Model
Build and train a TensorFlow model using the high-level Keras API.

In [ ]:
# build the model and train it
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu')) # relu is used for performance
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')) # softmax is used, because we only expect one class to occur per input
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae']) # ook adam ke proberen kan zijn dat het netwerk hiermee beter leert
# model.compile(optimizer='ftrl', loss='mse', metrics=['mae'])

history = model.fit(inputs_train, outputs_train, epochs=1500, batch_size=8, validation_data=(inputs_validate, outputs_validate))
model.summary()

2023-02-16 13:27:29.985823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 13:27:29.985947: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 13:27:29.988855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 13:27:29.988953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 13:27:29.989037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Epoch 1/1500


2023-02-16 13:27:29.989688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 13:27:30.093016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 13:27:30.093142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 13:27:30.093232: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so ret

52/52 [==============================] - 1s 3ms/step - loss: 0.1694 - mae: 0.2733 - val_loss: 0.1524 - val_mae: 0.2618
Epoch 2/1500
 1/52 [..............................] - ETA: 0s - loss: 0.1143 - mae: 0.2222

2023-02-16 13:27:31.297912: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


52/52 [==============================] - 0s 1ms/step - loss: 0.1356 - mae: 0.2467 - val_loss: 0.1449 - val_mae: 0.2575
Epoch 3/1500
52/52 [==============================] - 0s 1ms/step - loss: 0.1261 - mae: 0.2362 - val_loss: 0.1412 - val_mae: 0.2412
Epoch 4/1500
52/52 [==============================] - 0s 1ms/step - loss: 0.1245 - mae: 0.2270 - val_loss: 0.1382 - val_mae: 0.2255
Epoch 5/1500
52/52 [==============================] - 0s 1ms/step - loss: 0.1183 - mae: 0.2186 - val_loss: 0.1304 - val_mae: 0.2179
Epoch 6/1500
52/52 [==============================] - 0s 1ms/step - loss: 0.1146 - mae: 0.2072 - val_loss: 0.1218 - val_mae: 0.2185
Epoch 7/1500
52/52 [==============================] - 0s 1ms/step - loss: 0.1091 - mae: 0.2019 - val_loss: 0.1167 - val_mae: 0.2082
Epoch 8/1500
52/52 [==============================] - 0s 1ms/step - loss: 0.1049 - mae: 0.1929 - val_loss: 0.1116 - val_mae: 0.2042
Epoch 9/1500
52/52 [==============================] - 0s 1ms/step - loss: 0.1030 - mae: 0

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'g.', label="Training loss")
plt.plot(epochs, val_loss, 'b.', label="val loss")

plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.show()

#Run with Test Data
Put our test data into the model and plot the predictions

In [ ]:
# use the model to predict the test inputs
predictions = model.predict(inputs_test)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# print the predictions and the expected ouputs
print("predictions =\n", np.round(predictions, decimals=3))
print("actual =\n", outputs_test)

# sns.regplot(outputs_test, predictions)
#Predict
y_prediction = np.argmax(predictions, axis = 1)

y_test=np.argmax(outputs_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_prediction , normalize='pred')
disp =  ConfusionMatrixDisplay(confusion_matrix=result)
print(result)
disp.plot()
# Plot the predictions along with to the test data
# plt.clf()
# plt.title('Training data predicted vs actual values')
# plt.plot(inputs_test, outputs_test, 'b.', label='Actual')
# plt.plot(inputs_test, predictions, 'r.', label='Predicted')
# plt.show()

In [ ]:
array_test = [[33., 0.152, 0.83, 27., 18.]]
pred_test = model.predict(array_test)

print(pred_test)
class_index = np.argmax(pred_test)
print(class_index)
print(CLASSES[class_index])

In [ ]:
model.save('model_saved/')

In [ ]:
!tar chvfz notebook.tar.gz *